In [1]:
#import dependencies
from bs4 import BeautifulSoup as bs
from splinter import Browser
import os
import pandas as pd
import time
from urllib.parse import urlsplit

# NASA Mars News

In [2]:
#pointing to the directory where chromedriver exists
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser("chrome", **executable_path, headless = False)

In [3]:
#visiting the page
url = "https://mars.nasa.gov/news/"
browser.visit(url)

In [4]:
#using bs to write it into html
html = browser.html
soup = bs(html,"html.parser")

In [5]:
#scrape the latest News Title and Paragraph Text into variables
news_title = soup.find("div",class_="content_title").text
news_p = soup.find("div", class_="article_teaser_body").text
print(f"Title: {news_title}")
print(f"Paragraph: {news_p}")

Title: NASA's MRO Completes 60,000 Trips Around Mars
Paragraph: The orbiting spacecraft is also about to set a record for data relayed from the Martian surface.


# JPL Mars Space Images - Featured Image: Mars

In [6]:
#visiting the next page
url_mars_image = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url_mars_image)

In [7]:
#Getting the base url in order to get the path for this particular page
base_url = "{0.scheme}://{0.netloc}/".format(urlsplit(url_mars_image))
print(base_url)

https://www.jpl.nasa.gov/


In [8]:
#Design an xpath selector to grab the image. This will allow
#for me to direct to the same first picture in the search
#even if the picture changes. Used inspector to locate pathway.
xpath = "//*[@id=\"page\"]/section[3]/div/ul/li[1]/a/div/div[2]/img"

In [9]:
#Use splinter to click on the mars featured image
#to bring the full resolution image
results = browser.find_by_xpath(xpath)
mars_img = results[0]
mars_img.click()

In [10]:
#get image url using BeautifulSoup
html_image = browser.html
soup = bs(html_image, "html.parser")
mars_img_url = soup.find("img", class_="fancybox-image")["src"]
featured_image_url = base_url + mars_img_url
print(featured_image_url)

https://www.jpl.nasa.gov//spaceimages/images/largesize/PIA23222_hires.jpg


# Mars Weather Twitter Account

In [11]:
#get mars weather's latest tweet from the website
url_mars_weather = "https://twitter.com/marswxreport?lang=en"
browser.visit(url_mars_weather)

In [12]:
html_weather = browser.html
soup = bs(html_weather, "html.parser")
mars_weather_tweet = soup.find("p", class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text
print(mars_weather_tweet)

InSight sol 169 (2019-05-18) low -100.6ºC (-149.1ºF) high -17.6ºC (0.4ºF)
winds from the S at 4.6 m/s (10.2 mph) gusting to 15.5 m/s (34.7 mph)
pressure at 7.50 hPapic.twitter.com/QKbNMc35Ia


# Mars Facts

In [13]:
#identify the url to obtain the data and table
url_mars_facts = "https://space-facts.com/mars/"

In [14]:
#read the table into a variable. Used inspector to locate pathway.
facts_table = pd.read_html(url_mars_facts)
facts_table[0]

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [15]:
#convert table into a dataframe with column headers. Set Index to easily convert to html with table headers.
mars_facts_df = facts_table[0]
mars_facts_df.columns = ["Parameter", "Values"]
mars_facts_df.set_index(["Parameter"])

,Values
Parameter,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [16]:
#convert the dataframe into html and clean the format by removing \n
mars_html_table = mars_facts_df.to_html(index=False)
mars_html_table = mars_html_table.replace("\n", "")
mars_html_table

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th>Parameter</th>      <th>Values</th>    </tr>  </thead>  <tbody>    <tr>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <td>Mass:</td>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <td>Surface Temperature:</td>      <td>-153 to 20 °C</td>    </tr>    <tr>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <td>Recorded By:</td>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

# Mars Hemispheres

In [17]:
#select and go to the next website
url_mars_hemispheres = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url_mars_hemispheres)

In [18]:
#Getting the base url
hemispheres_base_url = "{0.scheme}://{0.netloc}/".format(urlsplit(url_mars_hemispheres))
print(hemispheres_base_url)

https://astrogeology.usgs.gov/


In [19]:
#create empty dictionary for titles and urls
hemispheres_dict = []

### Cerberus Hemisphere

In [20]:
#navigate to the large picture image and display the soup results to find picture
#use random time.sleep to mimic human clicks
results = browser.find_by_xpath( "//*[@id='product-section']/div[2]/div[1]/a/img").click()
time.sleep(2)
cerberus_open_click = browser.find_by_xpath( "//*[@id='wide-image-toggle']").click()
time.sleep(1)
cerberus_image = browser.html
soup = bs(cerberus_image, "html.parser")
#soup

In [21]:
#identify the picture url and save as a variable
cerberus_url = soup.find("img", class_="wide-image")["src"]
cerberus_img_url = hemispheres_base_url + cerberus_url
print(cerberus_img_url)

https://astrogeology.usgs.gov//cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg


In [22]:
#identify the picture title and save as a variable
cerberus_title = soup.find("h2",class_="title").text
print(cerberus_title)

Cerberus Hemisphere Enhanced


In [23]:
#append the dictionary to add the image title and the image url
cerberus = {"image title":cerberus_title, "image url": cerberus_img_url}
hemispheres_dict.append(cerberus)

### Schiaparelli Hemisphere

In [24]:
#select and go to the next website
url_mars_hemispheres = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url_mars_hemispheres)

In [25]:
#navigate to the large picture image and display the soup results to find picture
#use random time.sleep to mimic human clicks
results1 = browser.find_by_xpath( "//*[@id='product-section']/div[2]/div[2]/a/img").click()
time.sleep(2)
schiaparelli_open_click = browser.find_by_xpath( "//*[@id='wide-image-toggle']").click()
time.sleep(1)
schiaparelli_image = browser.html
soup = bs(schiaparelli_image, "html.parser")
#soup

In [26]:
#identify the picture url and save as a variable
schiaparelli_url = soup.find("img", class_="wide-image")["src"]
schiaparelli_img_url = hemispheres_base_url + schiaparelli_url
print(schiaparelli_img_url)

https://astrogeology.usgs.gov//cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg


In [27]:
#identify the picture title and save as a variable
schiaparelli_title = soup.find("h2",class_="title").text
print(schiaparelli_title)

Schiaparelli Hemisphere Enhanced


In [28]:
#append the dictionary to add the image title and the image url
schiaparelli = {"image title":schiaparelli_title, "image url": schiaparelli_img_url}
hemispheres_dict.append(schiaparelli)

### Syrtis Major Hemisphere

In [29]:
#select and go to the next website
url_mars_hemispheres = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url_mars_hemispheres)

In [30]:
#navigate to the large picture image and display the soup results to find picture
#use random time.sleep to mimic human clicks
results1 = browser.find_by_xpath( "//*[@id='product-section']/div[2]/div[3]/a/img").click()
time.sleep(2)
syrtis_major_open_click = browser.find_by_xpath( "//*[@id='wide-image-toggle']").click()
time.sleep(1)
syrtis_major_image = browser.html
soup = bs(syrtis_major_image, "html.parser")
#soup

In [31]:
#identify the picture url and save as a variable
syrtis_major_url = soup.find("img", class_="wide-image")["src"]
syrtis_major_img_url = hemispheres_base_url + syrtis_major_url
print(syrtis_major_img_url)

https://astrogeology.usgs.gov//cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg


In [32]:
#identify the picture title and save as a variable
syrtis_major_title = soup.find("h2",class_="title").text
print(syrtis_major_title)

Syrtis Major Hemisphere Enhanced


In [33]:
#append the dictionary to add the image title and the image url
syrtis_major = {"image title":syrtis_major_title, "image url": syrtis_major_img_url}
hemispheres_dict.append(syrtis_major)

### Valles Marineris Hemisphere

In [34]:
#select and go to the next website
url_mars_hemispheres = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url_mars_hemispheres)

In [35]:
#navigate to the large picture image and display the soup results to find picture
#use random time.sleep to mimic human clicks
results1 = browser.find_by_xpath( "//*[@id='product-section']/div[2]/div[4]/a/img").click()
time.sleep(2)
valles_marineris_open_click = browser.find_by_xpath( "//*[@id='wide-image-toggle']").click()
time.sleep(1)
valles_marineris_image = browser.html
soup = bs(valles_marineris_image, "html.parser")
#soup

In [36]:
#identify the picture url and save as a variable
valles_marineris_url = soup.find("img", class_="wide-image")["src"]
valles_marineris_img_url = hemispheres_base_url + syrtis_major_url
print(valles_marineris_img_url)

https://astrogeology.usgs.gov//cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg


In [37]:
#identify the picture title and save as a variable
valles_marineris_title = soup.find("h2",class_="title").text
print(valles_marineris_title)

Valles Marineris Hemisphere Enhanced


In [38]:
#append the dictionary to add the image title and the image url
valles_marineris = {"image title":valles_marineris_title, "image url": valles_marineris_img_url}
hemispheres_dict.append(valles_marineris)

In [39]:
#display the dictionary with all the image titles and image urls
hemispheres_dict

[{'image title': 'Cerberus Hemisphere Enhanced',
  'image url': 'https://astrogeology.usgs.gov//cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'image title': 'Schiaparelli Hemisphere Enhanced',
  'image url': 'https://astrogeology.usgs.gov//cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'image title': 'Syrtis Major Hemisphere Enhanced',
  'image url': 'https://astrogeology.usgs.gov//cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'image title': 'Valles Marineris Hemisphere Enhanced',
  'image url': 'https://astrogeology.usgs.gov//cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'}]